# 1 : Librairies Python

In [1]:
# 1 : Librairies et options
import datetime
import json
import os

import pandas as pd
import psycopg2
import requests
import tqdm
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.types import JSON, BigInteger, Integer

# 2 : Clés API et BDD :

In [2]:
# Informations API : https://weatherlink.github.io/v2-api/

load_dotenv()

# Clés API :
API_key = os.getenv("API_key")
API_secret = os.getenv("API_secret")
station_ID = os.getenv("station_ID")

# Paramètres de connexion à la base de données PostgreSQL en local :
host = os.getenv("host")
database = os.getenv("database")
user = os.getenv("user")
password = os.getenv("password")
nom_table = os.getenv("nom_table")

# 3 : Définitions :

In [3]:
def today_ts():
    """Récupération de la date du jour à 00h00 en TS pour utilisation comme
    date de fin avec l'API."""
    today = datetime.date.today()
    today_midnight = datetime.datetime.combine(today, datetime.time.min)
    end_date = int(today_midnight.timestamp())
    return end_date

In [4]:
def start_station():
    """Transformation de la date du début de la station en TS."""
    start_day = datetime.datetime(2021, 9, 29, 0, 0)
    start_day = int(start_day.timestamp())
    if_exists = "replace"  # informations pour la BDD
    return start_day, if_exists

In [5]:
def last_ts_bdd():
    """Récupération de la dernière TS enregistrée dans la base de données."""
    # Connexion à la base de données
    conn = psycopg2.connect(
        dbname=database,
        user=user,
        password=password,
        host=host,
    )
    cur = conn.cursor()

    # Exécution d'une requête SQL et récupération de la TS :
    cur.execute(f"SELECT ts FROM {nom_table} ORDER BY ts DESC LIMIT 1")
    data_extract = cur.fetchall()
    last_ts = pd.DataFrame(
        data_extract, columns=[desc[0] for desc in cur.description]
    ).values[0][0]
    if_exists = "append"  # informations pour la BDD

    # Fermeture du curseur et de la connexion
    cur.close()
    conn.close()

    return last_ts, if_exists

In [6]:
def start_api():
    """Choix de la TS de début à utiliser, en fonction de si la bdd est vide
    (historique) ou qu'elle contient déjà des données (routine)"""

    try:  # Présence d'une TS dans la table :
        start_date, if_exists = last_ts_bdd()

    except psycopg2.ProgrammingError:  # Gérer l'erreur connexion BDD
        start_date, if_exists = start_station()

    return start_date, if_exists

In [7]:
def one_day_data(start_date_api, end_date_api):
    """Récupération des données jour/jour via l'API et optention d'une DF."""
    # DataFrame historiques :
    df_ajout = pd.DataFrame()

    # Nb de jours à récupérer :
    nb_jours = int((end_date_api - start_date_api) / 86400)

    for i in tqdm.tqdm(range(nb_jours)):
        start_time = start_date_api + i * 86400
        end_time = start_time + 86400

        # Lien de la request :
        link = (
            f"https://api.weatherlink.com/v2/historic/{station_ID}?" # Base URL
            f"api-key={API_key}&"  # Clé API
            f"start-timestamp={start_time}&"  # Timestamp de début
            f"end-timestamp={end_time}"  # Timestamp de fin
        )

        headers = {"X-Api-Secret": API_secret}

        # Requête :
        r = requests.get(link, headers=headers, timeout=60)

        # Si la requête a réussi :
        if r.status_code == 200:
            # Lecture de la request en json :
            data = r.json()

            # Transformation en DF :
            df_jour = pd.DataFrame(data)
            df_jour = df_jour[["station_id", "sensors"]]

            # Récupération des valeurs se trouvant dans sensors :
            df_sensors = pd.json_normalize(data["sensors"][0]["data"])

            # Récupération des json sur une colonne :
            df_jour = pd.DataFrame(
                {
                    "station_id": data["station_id"],
                    "infos_json": data["sensors"][0]["data"],
                }
            )

            # Convertir les objets JSON en chaînes de caractères JSON :
            df_jour["infos_json"] = df_jour["infos_json"].apply(json.dumps)

            # Concat des données :
            df_jour = pd.concat([df_jour, df_sensors], axis=1)

            # Concaténation des données :
            df_ajout = pd.concat([df_ajout, df_jour], ignore_index=True)
        else:
            print(f"La requête {link} a échoué, code erreur : {r.status_code}")
    
    return df_ajout

In [8]:
def up_to_BDD(df_ajout, if_exists) :
    """Ajout des données dans la BDD."""
    # Connexion de la chaîne de connexion PostgreSQL :
    conn_str = f"postgresql://{user}:{password}@{host}/{database}"
    engine = create_engine(conn_str)

    # Définir les types de données pour chaque colonne :
    dtype = {"station_id": Integer(), "ts": BigInteger(), "infos_json": JSON}

    # Insérer le DataFrame dans la base de données PostgreSQL :
    df_ajout.to_sql(
        nom_table,
        engine,
        if_exists=if_exists,
        index=False,
        dtype=dtype,
    )

    # Fermeture de la connexion :
    engine.dispose()

# 4 : Récupération des données 

In [9]:
# Utilisation des définitions pour la récupération des données :
start_date_api, if_exists = start_api()
end_date_api = today_ts()
df_ajout = one_day_data(start_date_api, end_date_api)
up_to_BDD(df_ajout, if_exists)

  0%|                                                                                  | 1/942 [00:01<19:42,  1.26s/it]C:\Users\Johan\AppData\Local\Temp\ipykernel_5420\1577343197.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ajout = pd.concat([df_ajout, df_jour], ignore_index=True)
  9%|███████                                                                          | 82/942 [02:19<26:51,  1.87s/it]C:\Users\Johan\AppData\Local\Temp\ipykernel_5420\1577343197.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before

# 5 : Ouverture de la BDD

In [11]:
# Connexion à la base de données
conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour sélectionner les données de ma_table
cur.execute(f"SELECT * FROM {nom_table} LIMIT 5")

# Récupération des données dans une liste de tuples
data = cur.fetchall()

# Création d'un DataFrame à partir des données
df = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

df

,station_id,infos_json,ts,tz_offset,arch_int,rev_type,temp_out,temp_out_hi,temp_out_lo,temp_in,...,wind_run,deg_days_heat,deg_days_cool,solar_energy,uv_dose,thw_index,thsw_index,wet_bulb,night_cloud_cover,iss_reception
0,122495,"{""ts"": 1632942900, ""tz_offset"": 7200, ""arch_in...",1632942900,7200,300,2,56.1,56.2,56.1,74.6,...,0.0,0.030903,0.0,0.0,NaN,55.680000,52.794106,51.865425,0.5,None
1,122495,"{""ts"": 1632943200, ""tz_offset"": 7200, ""arch_in...",1632943200,7200,300,2,56.0,56.2,56.0,73.8,...,0.0,0.031250,0.0,0.0,NaN,55.580000,52.241997,51.771530,0.5,None
2,122495,"{""ts"": 1632943500, ""tz_offset"": 7200, ""arch_in...",1632943500,7200,300,2,56.2,56.2,56.0,71.6,...,0.0,0.030556,0.0,0.0,NaN,55.780003,52.441998,51.957924,0.5,None
3,122495,"{""ts"": 1632943800, ""tz_offset"": 7200, ""arch_in...",1632943800,7200,300,2,56.3,56.4,56.2,69.3,...,0.0,0.030208,0.0,0.0,NaN,55.880000,52.541996,52.050987,0.5,None
4,122495,"{""ts"": 1632944100, ""tz_offset"": 7200, ""arch_in...",1632944100,7200,300,2,57.7,57.7,56.3,67.4,...,0.0,0.025347,0.0,0.0,0.0,56.920002,53.537514,51.984350,0.5,None


In [12]:
import psycopg2

# Connexion à la base de données
conn = psycopg2.connect(dbname=database, user=user, password=password, host=host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour obtenir les informations de schéma de la table
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'historiquemeteo'")

# Récupération des données dans une liste de tuples
column_info = cur.fetchall()

# Affichage du format de chaque colonne
for column in column_info:
    print("La colonne ", column[0], "à pour format :", column[1])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

La colonne  et à pour format : double precision
La colonne  abs_press à pour format : double precision
La colonne  bar_noaa à pour format : double precision
La colonne  bar à pour format : double precision
La colonne  solar_rad_avg à pour format : double precision
La colonne  dew_point_out à pour format : double precision
La colonne  dew_point_in à pour format : double precision
La colonne  emc à pour format : double precision
La colonne  heat_index_out à pour format : double precision
La colonne  heat_index_in à pour format : double precision
La colonne  wind_chill à pour format : double precision
La colonne  wind_run à pour format : double precision
La colonne  deg_days_heat à pour format : double precision
La colonne  deg_days_cool à pour format : double precision
La colonne  solar_energy à pour format : double precision
La colonne  uv_dose à pour format : double precision
La colonne  thw_index à pour format : double precision
La colonne  thsw_index à pour format : double precision
